In [126]:
import wikipedia as wp
from wikipedia.exceptions import DisambiguationError, PageError
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import requests
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

In [10]:
to_ignore = set(stopwords.words('english'))
sentence = "A subset of recombination events results in crossovers, which create physical links known as chiasmata (singular: chiasma, for the Greek letter Chi (X)) between the homologous chromosomes."
sentence = ' '.join(map(lambda x: x if x not in to_ignore else '', sentence.split(' ')))
tokens = nltk.word_tokenize(sentence)
token_pos = nltk.pos_tag(tokens)
verb = "<VB|VBG|VBN|VBP|VBZ>*<RB|RBR|RBS>*"
word = "<NN|NNS|NNP|NNPS|JJ|JJR|JJS|RB|WP>"
preposition = "<IN>"
rel_pattern = "({}|{}{}|{}{}*{})+ ".format(verb, verb, preposition, verb, word, preposition)
grammar_long = '''REL_PHRASE: {%s}''' % rel_pattern
reverb_pattern = nltk.RegexpParser(grammar_long)
tree = list(reverb_pattern.parse(token_pos))
print(tree)

def iterate_over(tree):
    indices = list(filter(lambda y: y != -1, list(map(lambda x: tree.index(x) if type(x) == nltk.tree.Tree or x[1] == '.' else -1, tree))))
    return indices

indices = iterate_over(tree)
result = []

start = 0
for i in range(len(indices)):
    end = indices[i]
    sub_list = list(filter(lambda x: x[1] in ['NN', 'NNS', 'NNP'], tree[start:end]))
    last_noun = sub_list[-1][0] if len(sub_list) > 0 else None
    print(last_noun)
    print(tree[indices[i]])
    
    if(len(sub_list) > 0):
        result.append(last_noun)
        result.append(tree[end])
    start = end + 1

result

# iterate over
# last NNS/NN if it exists followed by whatever is before REL_PHARSE
# REL_PHRASE

# note how stage is a common noun that refers to a proper noun



[('A', 'DT'), ('subset', 'JJ'), ('recombination', 'NN'), ('events', 'NNS'), ('results', 'NNS'), ('crossovers', 'NNS'), (',', ','), Tree('REL_PHRASE', [('create', 'VB')]), ('physical', 'JJ'), ('links', 'NNS'), Tree('REL_PHRASE', [('known', 'VBN')]), ('chiasmata', 'NNS'), ('(', '('), ('singular', 'JJ'), (':', ':'), ('chiasma', 'NN'), (',', ','), ('Greek', 'JJ'), ('letter', 'NN'), ('Chi', 'NNP'), ('(', '('), ('X', 'NNP'), (')', ')'), (')', ')'), ('homologous', 'JJ'), ('chromosomes', 'NNS'), ('.', '.')]
crossovers
(REL_PHRASE create/VB)
links
(REL_PHRASE known/VBN)
chromosomes
('.', '.')


['crossovers',
 Tree('REL_PHRASE', [('create', 'VB')]),
 'links',
 Tree('REL_PHRASE', [('known', 'VBN')]),
 'chromosomes',
 ('.', '.')]

In [11]:
def parent_or_child(topic1, topic2): #returns 1 if topic1 is parent of topic2
    try:
        topic1_content = wp.page(topic1).content
        topic2_content = wp.page(topic2).content
        count2in1 = topic1_content.count(topic2) / len(topic1_content)
        count1in2 = topic2_content.count(topic1) / len(topic2_content)
        if count2in1 == 0 and count1in2 == 0:
            return 0
        elif count1in2 > count2in1:
            return 1
        else:
            return -1
    except DisambiguationError or PageError:
        return -2

# title = "meiosis"
# page = wp.page(title)
# content = ''.join(i for i in page.content if ord(i) < 128)
# sents = list(map(lambda x: x.strip(' '), content.split('.')))

# for link in page.links:
#     comp = parent_or_child(title, link)
#     if comp > 0:
#         print(title, "is parent of", link)
#     elif comp < 0:
#         print(link, "is parent of", title)
#     elif comp == 0:
#         print(title, link, "are siblings")

In [12]:
def get_triplet(sent):
    url = "http://www.newventify.com/rdf?sentence="
    r = requests.get(url + sent)
    try:
        rdf = r.json()['rdf']
        print("triplet gotten")
    except JSONDecodeError:
        return []
    return rdf

# for sent in sents:
#     trip = get_triplet(sent)
#     if title in trip:
#         print(trip)
#         print(sent)

# sents = [get_triplet(sent) for sent in sents]


In [13]:
import gensim
from gensim.utils import to_unicode

file_name = 'GoogleNews-vectors-negative300.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(file_name, binary=True, encoding='utf-8', unicode_errors = 'ignore') 
# # if you vector file is in binary format, change to binary=True




In [14]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in words:
        if word in model.wv.vocab:
            tokens.append(model[word])
            labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()


In [92]:
import re

# removes parantheses and spaces
def replace_parantheses(pre_string):
#     if pre_string.count('(') > pre_string.count(')'):
#         first_ind = pre_string.index('(')
#         return replace_parantheses(pre_string[:first_ind] + pre_string[first_ind + 1:])
    re_match = "\(.*\)" # anything besides ')'
    re_space = "\s\s+"
    return re.sub(re_space, ' ', re.sub(re_match, '', pre_string)).replace('\n', ' ')

def get_all_sents(title):
    try:
        page = wp.page(title)
    except DisambiguationError or PageError:
        return []
    content = ''.join(i for i in page.content if ord(i) < 128)
    con = replace_parantheses(content)
    sents = sent_tokenize(con)
    return sents, content

def get_valid_triplets(sents):
    triplets = [get_triplet(sent) for sent in sents]
    np_triplets = np.array(triplets)
    np_triplets = [np_triplets[i] if (np_triplets[i] != "").all() else [] for i in range(len(np_triplets))]
    return np.array(np_triplets).tolist()

def get_relevant_terms(title):
    try:
        page = wp.page(title)
    except DisambiguationError or PageError:
        return []
    return page.links # and those with high cosine scores from our word2vec

sents, content = get_all_sents("meiosis")
# sents[:10]

In [16]:
relevant_sents = [sent for sent in sents if "meiosis" in sent.lower()] # all sents with meiosis in it
relevant_triplets = get_valid_triplets(relevant_sents[20:30])

triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten


In [38]:
relevant_sents[20:30]

['\nThe term was linguistically corrected to "meiosis" by Koernicke, and by Pantel and De Sinety.',
 '\n\n\n== Phases ==\nMeiosis is divided into meiosis I and meiosis II which are further divided into Karyokinesis I and Cytokinesis I and Karyokinesis II and Cytokinesis II respectively.',
 'The preparatory steps that lead up to meiosis are identical in pattern and name to interphase of the mitotic cell cycle.',
 'This will take place during prophase I in meiosis.',
 '\nGrowth 2 phase: G2 phase as seen before mitosis is not present in meiosis.',
 'Interphase is followed by meiosis I and then meiosis II.',
 'Meiosis I separates homologous chromosomes, each still made up of two sister chromatids, into two daughter cells, thus reducing the chromosome number by half.',
 'During meiosis II, sister chromatids decouple and the resultant daughter chromosomes are segregated into four daughter cells.',
 'For diploid organisms, the daughter cells resulting from meiosis are haploid and contain only

In [19]:
parent_words = ["includes", "contains"]
sibling_words = ["causes", "caused"]
child_words = ["suchas"]

def max_sim(relation, pred):
    if relation in ["parent", "sibling", "child"]:
        return sum([model.similarity(pred, word) for word in globals()["{}_words".format(relation)]]) / len(globals()["{}_words".format(relation)])

def best_relation(pred):
    return max(["parent", "sibling", "child"], key = lambda x: max_sim(x, pred))

def subject_relation(triplet):
    return best_relation(triplet[1])
    
for triplet in relevant_triplets:
    if len(triplet) > 0:
        pred = triplet[1]
        print(triplet)
        print(triplet[0], "is of relation", best_relation(triplet[1]), "to", triplet[2])


# THIS WORKS WELL DIFFERENTIATING BETWEEN HIGHER/LOWER LEVEL AND SIBLING LEVEL
# BUT NOT SO WELL BETWEEN PARENT AND CHILD


len(sents)

['term' 'corrected' 'meiosis']
term is of relation sibling to meiosis
['steps' 'lead' 'meiosis']
steps is of relation sibling to meiosis
['This' 'take' 'place']
This is of relation parent to place
['Growth' 'seen' 'mitosis']
Growth is of relation sibling to mitosis
['Interphase' 'followed' 'I']
Interphase is of relation sibling to I
['I' 'separates' 'chromosomes']
I is of relation parent to chromosomes
['meiosis' 'segregated' 'daughter']
meiosis is of relation child to daughter
['organisms' 'resulting' 'meiosis']
organisms is of relation sibling to meiosis
['species' 'known' 'interkinesis']
species is of relation parent to interkinesis


247

In [63]:
import os
import subprocess

def get_openie_triplets(sents):
    openie_triplets = []
    for i in range(len(sents)):
        sent = sents[i]
        cmd = "cd Stanford-OpenIE-Python; echo \“" + sent + "\” > samples.txt; python main.py -f samples.txt"
        trip_array = os.popen(cmd).read().split('\n')
        trip_array = np.array(trip_array)[np.array(trip_array) != ''].tolist()
        openie_triplets.append(trip_array)
    return openie_triplets

triplets = get_openie_triplets(sents[10:20])

[['cohesion | is released During | Meiosis II', 'cohesion | is | released'],
 [],
 ['cell | produce | ovum'],
 [],
 [],
 ['alternating cycles | Thus enable reproduction with | generations',
  'cycles | enable with | generations',
  'cycles | enable reproduction with | generations',
  'alternating cycles | Thus enable with | successive generations',
  'cycles | Thus enable | sexual reproduction',
  'alternating cycles | enable | reproduction',
  'maintaining | number of | chromosomes',
  'alternating cycles | enable reproduction with | generations',
  'alternating cycles | enable with | successive generations',
  'alternating cycles | Thus enable reproduction with | successive generations',
  'cycles | enable reproduction with | successive generations',
  'alternating cycles | enable | sexual reproduction',
  'alternating cycles | Thus enable | sexual reproduction',
  'cycles | Thus enable reproduction with | generations',
  'cycles | Thus enable reproduction with | successive generatio

In [93]:
# for sent in sents
#     get triplet
#     get openie_triplets
#     get expanded subject, object from openie_triplet
#     find highest predicate score of any category of valid openie_triplets

# for i in range((len(sents) + 10) // 10):
i = 1
batch = sents[10 * i: min(len(sents), 10 * (i + 1))]
triplets = get_valid_triplets(batch)
openie_triplets = get_openie_triplets(batch)

triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten
triplet gotten


([['cell', 'undergoes', 'DNA'],
  ['each', 'leading', 'connections'],
  ['division', 'separate', 'daughter'],
  ['cells', 'proceed', 'division'],
  ['sister', 'produce', 'total'],
  ['animals', 'employ', 'variation'],
  ['recombination', 'are', 'distinct'],
  ['gamete', 'include', 'assortment'],
  ['diversity', 'resulting', 'reproduction'],
  ['Meiosis', 'used', 'eukaryotes']],
 [['homolog | consists of | two sister chromatids',
   'homolog | now consists of | two identical sister chromatids',
   'homolog | now consists of | two sister chromatids',
   'homolog | consists of | two identical sister chromatids'],
  ['set | is with | other DNA'],
  ['homologs | are segregated In | meiotic division',
   'homologs | are | segregated',
   'homologs | are segregated In | first division',
   'homologs | are segregated In | first meiotic division',
   'homologs | separate | daughter cells',
   'homologs | are segregated In | division',
   'homologs | separate daughter cells by | spindle apparatu

In [104]:
triplets[6], openie_triplets[6]

# this is surprising, very few subject, object pair from triplet is contained entirely in its openie_triplets counterpart
# but its clear openie_triplets outperform triplets

# weight an idf for openie_triplets predicates in order to calculate score

def predicate_idf(pred):
    tokens = pred.split(' ')
    

(['recombination', 'are', 'distinct'],
 ['individual chromatid | can consist of | new combination of maternal DNA',
  'individual chromatid | can consist of | new combination',
  'individual chromatid | can consist of | new combination of DNA',
  'chromatid | can consist Because of | recombination',
  'chromatid | can consist of | new combination',
  'chromatid | can consist of | new combination of DNA',
  'individual chromatid | can consist Because of | recombination',
  'chromatid | can consist of | combination of DNA',
  'chromatid | can consist of | combination',
  'individual chromatid | can consist of | combination',
  'individual chromatid | can consist of | combination of DNA',
  'chromatid | can consist of | new combination of maternal DNA',
  'individual chromatid | can consist of | combination of maternal DNA',
  'chromatid | can consist | resulting',
  'individual chromatid | can consist | resulting',
  'chromatid | can consist of | combination of maternal DNA'])

In [127]:
# Dictionary(' '.join(sents))

word_tokenize(sents[0])

['Meiosis', 'between', 'the', 'homologous', 'chromosomes', '.']